In [ ]:
import numpy as np


import matplotlib

matplotlib.rcParams["animation.embed_limit"] = 128
import matplotlib.pyplot as plt
import IPython

my_cmap = plt.get_cmap("plasma")

In [ ]:
def lorenz(dt, coordinates, parameters=(10, 12, 8./3.), scale=1.0):
    """
    
    """
    
    sigma = parameters[0]
    rho = parameters[1]
    beta = parameters[2]
    
    x, y, z = coordinates[...,0:1], coordinates[...,1:2], coordinates[...,2:3]
    
    while len(dt.shape) < len(x.shape):
        dt = dt[:,None]
        
    new_x = x + dt * (scale*(sigma*(y-x)))
    new_y = y + dt * (scale*(rho*x - x*z - y))
    new_z = z + dt * (scale*(x*y - beta*z))
    
    #identify boundary edges
    identify = lambda xyz, boundary: np.sign(xyz)*(np.abs(xyz) % boundary)
    bedge = 30
    new_x = identify(new_x, bedge)
    new_y = identify(new_y, bedge)
    new_z = identify(new_z, bedge)
    
    new_coordinates = np.append(np.append(new_x, new_y, axis=-1),\
        new_z, axis=-1)

    return new_coordinates

def plot_xy_projection(coordinates):
    
    global subplot_xy
    
    fig, ax = plt.subplots(1,1, figsize=(4,4))
    
    subplot_xy = ax.scatter(coordinates[:,0], coordinates[:,2], alpha=0.5)
    ax.axis([-20,20,0,20])
    
    return fig, ax

def plot_xyz_projection(coordinates):
    
    global subplot_xyz
    
    fig = plt.figure(figsize=(4,4))
    ax = fig.add_subplot(projection='3d')
    
    subplot_xyz = ax.scatter(*coordinates.T, alpha=0.5)
    
    ax.set_xlim(-12,12)
    ax.set_ylim(-12,12)
    ax.set_zlim(-2,22)
    
    return fig, ax

def update_xyz_projection(t):
    
    global subplot_xyz
    global coordinates
    global dt
        
    #subplot_xyz.set_offsets(coordinates)
    subplot_xyz._offsets3d = (coordinates[:,0], coordinates[:,1], coordinates[:,2]) 
    coordinates = lorenz(dt, coordinates)
    
    
def update_xy_projection(t):
    
    global subplot_xy
    global dt    
    global coordinates
        
    subplot_xyz.set_offsets(np.append(coordinates[:,0:1], coordinates[:,2:3], axis=-1))
    
    coordinates = lorenz(dt, coordinates)
    
def plot_lorenz_map(dt_display, x):
    
    global subplot_x
    global subplot_y
    global subplot_z
    
    my_cmap = plt.get_cmap("viridis")
    
    fig, ax = plt.subplots(1,1, figsize=(6,4))
    my_step = 0
    
    subplot_x = ax.scatter(dt_display.squeeze(), x[...,0].T.ravel(),
            marker="x", s=8, color=my_cmap(8), alpha=0.53, label="x")
    subplot_y = ax.scatter(dt_display.squeeze(), x[...,1].T.ravel(),
            marker="s", s=8, color=my_cmap(128), alpha=0.53, label="y" )
    subplot_z = ax.scatter(dt_display.squeeze(), x[...,2].T.ravel(),
            marker="o", s=8, color=my_cmap(256), alpha=0.53, label="z" )

    plt.xlabel("time step $\Delta t$", fontsize=20)
    plt.ylabel(f"x,y,z at t={my_step}", fontsize=20)
    plt.title(f"Lorenz values at t={my_step}", fontsize=24)
    plt.legend()
    plt.tight_layout()
    
    return fig, ax

def update_lorenz_map(t):
    
    global subplot_x
    global subplot_y
    global subplot_z
    global dt_display
    global dt
    global coordinates
    global fig
    global ax
    
    xx = coordinates[...,0]
    yy = coordinates[...,1]
    zz = coordinates[...,2]
            
    subplot_x.set_offsets(\
            np.append(dt_display, xx.T.ravel()[:,None], axis=-1))             
    subplot_y.set_offsets(\
            np.append(dt_display, yy.T.ravel()[:,None], axis=-1))
    subplot_z.set_offsets(\
            np.append(dt_display, zz.T.ravel()[:,None], axis=-1))

    ax.set_title(f"Lorenz values at t={t}")
                          
    coordinates = lorenz(dt, coordinates)

In [ ]:
dts = np.array([0.03, 0.015])
number_steps = 4096
labels = ["a", "b", "c", "d"]

# initial conditions
coordinates_0 = np.array([[2.,1.,1.]])
# each 
xyzs = [] 

fig = plt.figure(figsize=(8,4.5))

gs = fig.add_gridspec(1, 2)
axes = []
axes.append(fig.add_subplot(gs[0,0], projection='3d'))
axes.append(fig.add_subplot(gs[0,1], projection='3d'))

for idx in range(2):

    dt = dts[idx % 2:idx % 2+1]
    xyz = coordinates_0*1.0
    
    if idx == 2:
        scale = 0.5
    elif idx == 3:
        scale = 2.0
    else: 
        scale = 1.0
        
    my_steps = int(number_steps * (dts[1] / dt) / scale) 
        
    for my_step in range(my_steps):
        xyz = np.append(xyz, lorenz(dt, xyz[-1:], scale=scale), axis=0)

    colors = [1.0 - np.array(my_cmap(0.5+0.5*elem/len(xyz))[0:3]) for elem in range(len(xyz))]

    for ii in range(xyz.shape[0]-1):
        axes[idx].plot(*xyz[ii:ii+2].T, lw=0.75, alpha=0.75, color=colors[ii])


    axes[idx].scatter(*xyz[0:1].T, marker="s", s=50, edgecolor="k", color=colors[0])
    axes[idx].scatter(*xyz[-1:].T, marker="o", s=50, edgecolor="w", color=colors[-1])
    axes[idx].set_title(f"Step size $\\Delta t$ = {dt[0]:.3f}")
                        
fig.suptitle("Lorenz system: $\\sigma = 10$, $\\rho=12$, $\\beta=\\frac{8}{3}$", \
        fontsize=20)

for ax in axes:
    ax.set_xlim(-10, 10)
    ax.set_ylim(-10, 10)
    ax.set_zlim(0, 20)

axes[0].text(-10.0, -15.0, -13.5, "a", fontsize=48)
axes[1].text(-10.0, -15.0, -13.5, "b", fontsize=48)

#plt.tight_layout()
plt.savefig("lorenz_dt.png")
plt.show()

In [ ]:
for my_step in range(8192):
    xyz = np.append(xyz, lorenz(dt, xyz[-1:], scale=scale), axis=0)

# show steady state values
xyz[-10:]

In [ ]:
dts = np.array([0.03, 0.015])
number_steps = 208
labels = ["a", "b", "c", "d"]

# initial conditions
coordinates_0 = np.array([[2.,1.,1.]])
# each 
xyzs = [] 

fig = plt.figure(figsize=(8,8))

gs = fig.add_gridspec(2, 2)
axes = []
axes.append(fig.add_subplot(gs[0,0], projection='3d'))
axes.append(fig.add_subplot(gs[0,1], projection='3d'))
axes.append(fig.add_subplot(gs[1,0], projection='3d'))
axes.append(fig.add_subplot(gs[1,1], projection='3d'))

for idx in range(4):

    dt = dts[idx % 2:idx % 2+1]
    xyz = coordinates_0*1.0
    
    if idx == 2:
        scale = 0.5
    elif idx == 3:
        scale = 2.0
    else: 
        scale = 1.0
        
    my_steps = int(number_steps * (dts[1] / dt) / scale) 
        
    for my_step in range(my_steps):
        xyz = np.append(xyz, lorenz(dt, xyz[-1:], scale=scale), axis=0)

    colors = [1.0 - np.array(my_cmap(0.5+0.5*elem/len(xyz))[0:3]) for elem in range(len(xyz))]

    for ii in range(xyz.shape[0]-1):
        axes[idx].plot(*xyz[ii:ii+2].T, lw=0.75, alpha=0.75, color=colors[ii])


    axes[idx].scatter(*xyz[0:1].T, marker="s", s=50, edgecolor="k", color=colors[0])
    axes[idx].scatter(*xyz[-1:].T, marker="o", s=50, edgecolor="w", color=colors[-1])
    axes[idx].set_title(f"Step size $\\Delta t$ = {dt[0]:.3f}")
                        
fig.suptitle("Lorenz system \n$\\sigma = 10$, $\\rho=12$, $\\beta=\\frac{8}{3}$")

for ax in axes:
    ax.set_xlim(-10, 10)
    ax.set_ylim(-10, 10)
    ax.set_zlim(0, 20)
    
plt.tight_layout()
plt.show()

In [ ]:
max_steps = 2**15 #8192
number_samples = 16

number_dts = 100
max_dt = .1
ddt = max_dt / number_dts
min_dt = ddt

dt = np.arange(min_dt, max_dt+ddt, ddt)
coordinates =  np.random.rand(dt.shape[0], number_samples, 3) * 60 - 30
coordinates.shape
c_0 = 1.0 * coordinates
# concatenate r vector 
dt_display = np.append(dt, dt, axis=0)

while dt_display.shape[0] < dt.shape[0]*number_samples:
    dt_display = np.append(dt_display, dt_display, axis=0)

dt_display = dt_display[:,None]

fig, ax = plot_lorenz_map(dt_display, coordinates)
plt.show()

for ii in range(max_steps):
    coordinates = lorenz(dt, coordinates)
    
    
fig, ax = plot_lorenz_map(dt_display, coordinates)
ax.set_title(f"Lorenz values at t={ii}")
ax.set_ylabel(f"x,y,z values at t={ii}")
plt.show()

In [ ]:
# animation
max_steps = 1024
number_samples = 8

number_dts = 100
max_dt = 0.1 #1.0
ddt = max_dt / number_dts
min_dt = ddt

dt = np.arange(min_dt, max_dt+ddt, ddt)
coordinates =  np.random.rand(dt.shape[0], number_samples, 3) * 60 - 30
coordinates.shape
c_0 = 1.0 * coordinates
# concatenate r vector 
dt_display = np.append(dt, dt, axis=0)

while dt_display.shape[0] < dt.shape[0]*number_samples:
    dt_display = np.append(dt_display, dt_display, axis=0)

dt_display = dt_display[:,None]

fig, ax = plot_lorenz_map(dt_display, coordinates)
plt.show()

IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_lorenz_map, \
        frames=max_steps, interval=100).to_jshtml())

In [ ]:
max_steps = 124
number_samples = 96

coordinates =  np.random.randn(number_samples, 3) * 2 #+ np.array([2,1,1.0])
print(coordinates.shape)

dt = np.array([0.015])
fig, ax  = plot_xyz_projection(coordinates)
plt.show()

for ii in range(max_steps*3):
    coordinates = lorenz(dt, coordinates)

IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_xyz_projection, \
        frames=max_steps, interval=100).to_jshtml())

In [ ]:
max_steps = 1024
number_samples = 96

coordinates =  np.random.randn(number_samples, 3) * 2 #+ np.array([2,1,1.0])
print(coordinates.shape)

dt = np.array([0.03])
fig, ax  = plot_xyz_projection(coordinates)
plt.show()

for ii in range(max_steps*3):
    coordinates = lorenz(dt, coordinates)

IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_xyz_projection, \
        frames=max_steps, interval=100).to_jshtml())